In [1]:
from tensorflow.keras.callbacks import TensorBoard

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
import pickle
import time

In [2]:
NAME = "Cats-vs-dogs-CNN"

In [6]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

In [7]:
model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [8]:
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [9]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )

In [ ]:
model.fit(X, y,
          batch_size=32,
          epochs=3,
          validation_split=0.3,
          callbacks=[tensorboard])

In [ ]:
# After having run this, you should have a new directory called logs. We can visualize the initial results from this directory using tensorboard now. Open a console, change to your working directory, and type: tensorboard --logdir=logs/. You should see a notice like: TensorBoard 1.10.0 at http://H-PC:6006 (Press CTRL+C to quit) where "h-pc" probably is whatever your machine's name is. 

In [ ]:
# Now we can see how our model did over time. Let's change some things in our model. To begin, we never added an activation to our dense layer. Also, let's try a smaller model overall:

In [10]:
NAME = "Cats-vs-dogs-64x2-CNN"

In [11]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

In [12]:
model = Sequential()

model.add(Conv2D(64, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [13]:
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'],
              )

In [ ]:
model.fit(X, y,
          batch_size=32,
          epochs=10,
          validation_split=0.3,
          callbacks=[tensorboard])

In [ ]:
# Among other things, I also changed the name to NAME = "Cats-vs-dogs-64x2-CNN". Don't forget to do this, or you'll append to your previous model's logs instead by accident and it wont look too good. Let's check TensorBoard now:

In [ ]:
# Looking better! Immediately, however, you might notice the shape of validation loss. Loss is the measure of error, and it clearly looks like, after our 4th epoch, things began to sour. Interestingly enough, our validation accuracy still continued to hold, but I imagine it would eventually begin to fall. It's much more likely that the first thing to suffer will indeed be your validation loss. This should alert you that you're almost certainly beginning to over-fit. The reason why this happens is the model is constantly trying to decrease your in-sample loss. At some point, rather than learning general things about the actual data, the model instead begins to just memorize input data. If you let this continue, yes your "accuracy" in-sample will rise, but your out of sample, and any new data you attempt to feed the model, will perform poorly.